<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Stochastic-Optimization" data-toc-modified-id="Stochastic-Optimization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Stochastic Optimization</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Data-Imports" data-toc-modified-id="Data-Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Data Imports</a></span></li><li><span><a href="#Functions-for-Variables-and-Constraints" data-toc-modified-id="Functions-for-Variables-and-Constraints-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Functions for Variables and Constraints</a></span></li><li><span><a href="#Model-Creation" data-toc-modified-id="Model-Creation-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Model Creation</a></span></li></ul></li></ul></li><li><span><a href="#Robust-Optimization" data-toc-modified-id="Robust-Optimization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Robust Optimization</a></span></li><li><span><a href="#Benders-Decomposition-with-Stochastic-programming" data-toc-modified-id="Benders-Decomposition-with-Stochastic-programming-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Benders Decomposition with Stochastic programming</a></span></li></ul></div>

# Stochastic Optimization


Stochastic optimization model used to solve the two-stage problem

In [1]:
using JuMP,HiGHS,DataFrames, GLPK
using CSV
using DataFrames

### Data Imports

In [2]:
file_prefix = "24_buses_"

# Import data about buses
df_buses = CSV.read(string("data/",file_prefix,"paperdata.csv"), DataFrame,types=Dict("C" => String,"I" => String, "C_cur" => String, "Lambda" => String))
#Import demand data
df_demands = CSV.read(string("data/",file_prefix,"demand.csv"), DataFrame)
# Import the realized wind energy
df_realized_energy_scenarios = CSV.read(string("data/15_zones_scenarios.csv"), DataFrame)
# Import the remaining data
df_other = CSV.read(string("data/",file_prefix,"other_data.csv"), DataFrame)


Row,Omega,Pi
,Int64,Float64
1,1,0.1
2,2,0.1
3,3,0.1
4,4,0.1
5,5,0.1
6,6,0.1
7,7,0.1
8,8,0.1
9,9,0.1


In [3]:
"""
Transforms indices into range for lists of objects (such as generation units)
Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)
Input of missing cells is translated to no values (i.e. empty array)
Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0" return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end

"""
Transforms a range and an array into one dictionary
"""
function array_to_dict(keys::AbstractVector{T}, values::AbstractVector{S}) where {T,S}
    # Check if arrays have identical length
    if length(keys) != length(values)
        throw(ArgumentError("Array length must be identical"))
    end

    # Initialize empty dictionary
    result_dict = Dict{T,S}()

    # Add key value pairs
    for i in 1:length(keys)
        result_dict[keys[i]] = values[i]
    end

    return result_dict
end


array_to_dict

In [4]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for single_node in eachrow(df_buses)
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => string_to_array(string.(singleNode.I)),
        :Q => string_to_array(string.(singleNode.Q)),
        :J => string_to_array(string.(singleNode.J)),
        :Λ => string_to_array(string.(singleNode.Lambda)),
        :C_save => string_to_array(string(singleNode.C)) ,
        :C_RU_save => string_to_array(string(singleNode.C_RU)),
        :C_RD_save => string_to_array(string(singleNode.C_RD)),
        :C_U_save => string_to_array(string(singleNode.C_U)),
        :C_D_save => string_to_array(string(singleNode.C_D)),
        :V_LOL_save => string_to_array(string(singleNode.V_LOL)),
        :C_cur_save => string_to_array(string(singleNode.C_cur)),
        :P_max_save => string_to_array(string(singleNode.P_max)),
        :b_save => string_to_array(string(singleNode.b)),
        :LC_Max_save => string_to_array(string(singleNode.LC_Max)),
        :W_s_save => string_to_array(string(singleNode.W_s)),
        :W_d_max_save => string_to_array(string(singleNode.W_d_max)),
        :W_s => Dict(),
        :W_d_min => Dict(),
        :W_d_max => Dict(),
        :L_save => Dict(),
        :C => Dict(),
        :C_RU => Dict(),
        :C_RD => Dict(),
        :C_U => Dict(),
        :C_D => Dict(),
        :P_max => Dict(),
        :C_cur => Dict(),
        :b => Dict(),
        :LC_Max => Dict(),
        :V_LOL => Dict(),
        :Ω => 0,
        :T => 1:1,
    )    
    # Assign the dictionary to the current node key
    data[single_node.Node] = data_node
end

println(data[2][:Q])
#print(data[1][:W_s])


Float64[]


In [5]:
for singleNode in keys(data)
    # Create a dictionary for the current node with all other variables
    data[singleNode][:C] = array_to_dict(data[singleNode][:I], data[singleNode][:C_save])
    data[singleNode][:C_RU] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RU_save])
    data[singleNode][:C_RD] = array_to_dict(data[singleNode][:I], data[singleNode][:C_RD_save])
    data[singleNode][:C_U] = array_to_dict(data[singleNode][:I], data[singleNode][:C_U_save])
    data[singleNode][:C_D] = array_to_dict(data[singleNode][:I], data[singleNode][:C_D_save])
    data[singleNode][:P_max] = array_to_dict(data[singleNode][:I], data[singleNode][:P_max_save])
    data[singleNode][:C_cur] = array_to_dict(data[singleNode][:Q], data[singleNode][:C_cur_save])
    data[singleNode][:b] = array_to_dict(data[singleNode][:Λ], data[singleNode][:b_save])
    data[singleNode][:LC_Max] = array_to_dict(data[singleNode][:Λ], data[singleNode][:LC_Max_save])
    data[singleNode][:V_LOL] = array_to_dict(data[singleNode][:J], data[singleNode][:V_LOL_save])
    data[singleNode][:W_s]= array_to_dict(data[singleNode][:Q], data[singleNode][:W_s_save])
    data[singleNode][:W_d_max]= array_to_dict(data[singleNode][:Q], data[singleNode][:W_d_max_save])
end

In [6]:
#Create dictionary for all demands dependent on node and load -> array with different times
known_loads = []
demands = Dict()
default_periods = 1
for single_node in eachrow(df_demands)
    data_node = Dict(
        :L => string_to_array(string(single_node.Demands)),
    )    
    if  single_node.Node ∉ known_loads
        demands = Dict()
    end
    push!(known_loads, single_node.Node)

    demands[single_node.Loads] = data_node
    data[single_node.Node][:L] = demands
    
    data[single_node.Node][:T] = 1 : single_node.Periods
    if default_periods < single_node.Periods
        default_periods = single_node.Periods
    end
end

#Fill periods of Nodes without demands -> Set Default Value
for single_node in keys(data)
    #if data[single_node][:T] === nothing
        data[single_node][:T] = minimum(1):maximum(default_periods)
    #end
end


In [7]:
#create dictionary with all scenarios
scenarios = Dict()
for single_node in eachrow(df_other)
    data_node = Dict(
        :π => single_node.Pi,
    )
    scenarios[single_node.Omega] = data_node
end


# Convert float to int
nodes_with_wind_turbines = []
for value in keys(data)
    data[value][:Λ] = round.(Int, data[value][:Λ])
    if length(data[value][:Q]) > 0
        push!(nodes_with_wind_turbines, value)
    end
end

#create dictionary with all realized Energy from wind turbine dependent on node, wind turbine, scenario -> array with different times
wind_energy = Dict()
data_omega = Dict()
for single_node in eachrow(df_realized_energy_scenarios)
    data_node = Dict(
        :W_realized => string_to_array(string(single_node.Energy)),
    )
    if ! haskey(wind_energy, single_node.Q)
        data_omega = Dict()
    end
    data_omega[single_node.Omega] = data_node
    wind_energy[single_node.Q] = data_omega
    data[nodes_with_wind_turbines[single_node.Q]][:wind_energy] = wind_energy
end

#wind_energy must be converted into max values in MW
for n in keys(data)
    for q in data[n][:Q]
        for ω in keys(scenarios)
            for t in data[n][:T]
                data[n][:wind_energy][q][ω][:W_realized][t] = data[n][:wind_energy][q][ω][:W_realized][t] * 300
            end
        end
    end
end

### Functions for Variables and Constraints

In [8]:
# Initializes variables
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0) #scheduled wind power generation at turbine q
    delta=@variable(model, delta[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]])
    delta_scenario=@variable(model, delta_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]]) #voltage angle in case 𝜔
    PF=@variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #power flow
    PF_scenario=@variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]) #power flow in case 𝜔
    
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :delta => delta,
        :delta_scenario => delta_scenario,
        :PF => PF,
        :PF_scenario => PF_scenario,
    )    
    return vars
end;

In [9]:
# Initializes constraints
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
    c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]

    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:delta][n, ℓ, t] - vars[:delta][ℓ, n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
    c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:delta_scenario][n, ℓ, ω, t] - vars[:delta_scenario][ℓ, n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]
    
    c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:wind_energy][q][ω][:W_realized][t]
    c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]

    Reference_Node[t in data[1][:T]], vars[:delta][1, :, t] .== 0
    Reference_Node_Scenario[ω in keys(scenarios), t in data[1][:T]], vars[:delta_scenario][1, :, ω, t] .== 0


    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:wind_energy][q][ω][:W_realized][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0
    end)
end;

In [10]:
# Initializes objective function
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

### Model Creation

In [11]:
# Create and print model
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
#print(model)

13.32 P[1,1.0,1] + 15 R_U[1,1.0,1] + 14 R_D[1,1.0,1] + 13.32 P[1,1.0,2] + 15 R_U[1,1.0,2] + 14 R_D[1,1.0,2] + 13.32 P[1,1.0,3] + 15 R_U[1,1.0,3] + 14 R_D[1,1.0,3] + 13.32 P[1,1.0,4] + 15 R_U[1,1.0,4] + 14 R_D[1,1.0,4] + 13.32 P[1,1.0,5] + 15 R_U[1,1.0,5] + 14 R_D[1,1.0,5] + 13.32 P[1,1.0,6] + 15 R_U[1,1.0,6] + 14 R_D[1,1.0,6] + 13.32 P[1,1.0,7] + 15 R_U[1,1.0,7] + 14 R_D[1,1.0,7] + 13.32 P[1,1.0,8] + 15 R_U[1,1.0,8] + 14 R_D[1,1.0,8] + 13.32 P[1,1.0,9] + 15 R_U[1,1.0,9] + 14 R_D[1,1.0,9] + 13.32 P[1,1.0,10] + 15 R_U[1,1.0,10] + 14 R_D[1,1.0,10] + [[...9204 terms omitted...]] + W_spill[21,9.0,3,19] + W_spill[21,9.0,3,20] + W_spill[21,9.0,3,21] + W_spill[21,9.0,3,22] + W_spill[21,9.0,3,23] + W_spill[21,9.0,3,24] + W_spill[21,9.0,1,1] + W_spill[21,9.0,1,2] + W_spill[21,9.0,1,3] + W_spill[21,9.0,1,4] + W_spill[21,9.0,1,5] + W_spill[21,9.0,1,6] + W_spill[21,9.0,1,7] + W_spill[21,9.0,1,8] + W_spill[21,9.0,1,9] + W_spill[21,9.0,1,10] + W_spill[21,9.0,1,11] + W_spill[21,9.0,1,12] + W_spill[21,

In [12]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27624 rows, 45336 cols, 103464 nonzeros
9480 rows, 19008 cols, 51048 nonzeros
8736 rows, 17736 cols, 48072 nonzeros
Presolve : Reductions: rows 8736(-44472); columns 17736(-28752); elements 48072(-85320)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.9557641259e+03 Ph1: 2377(2610); Du: 1253(1955.76) 0s
      10021     1.6619527007e+05 Pr: 0(0); Du: 0(5.9508e-13) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 10021
Objective value     :  1.6619527007e+05
HiGHS run time      :          0.80


In [13]:

# Print the values of the variables
# println("Variable values:")
# println("P:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("P[$n, $i, $t]: ", JuMP.value.(vars[:P][n, i, t]))
# end

# println("\nR_U:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_U[$n, $i, $t]: ", value(vars[:R_U][n,i, t]))
# end

# println("\nR_D:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_D[$n, $i, $t]: ", value(vars[:R_D][n,i, t]))end

# println("\nL_Shed:")
# for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
#     println("L_Shed[$n, $j, $ω, $t]: ", value(vars[:L_Shed][n,j, ω, t]))
# end

# println("\nW_spill:")
# for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
#     println("W_spill[$n, $q, $ω, $t]: ", value(vars[:W_spill][n,q, ω, t]))
# end

# println("\nW_s:")
# for n in keys(data), q in data[n][:Q], t in data[n][:T]
#     println("W_s[$n, $q, $t]: ", value(vars[:W_s][n,q, t]))
# end

# println("\nPF[n, ℓ, t]:")
# for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
#     println("PF[$n, $ℓ, $t]: ", value(vars[:PF][n, ℓ, t]))
# end

# println("\nPF_scenario[n, ℓ, ω, t]:")
# for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
#     println("PF_scenario[$n, $ℓ, $ω, $t]: ", value(vars[:PF_scenario][n, ℓ, ω, t]))
# end

# println("\nr_U[n,i, ω, t]:")
# for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#     println("r_U[$n, $i, $ω, $t]: ", value(vars[:r_U][n,i, ω, t]))
# end

# println("\nr_D[n,i, ω, t]:")
# for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#     println("r_D[$n, $i, $ω, $t]: ", value(vars[:r_D][n,i, ω, t]))
# end

# println("\nData values:")

# println("\nW_realized:")
# for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
#     println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
# end

# println("\nLoad:")
# for n in keys(data), j in data[n][:J], t in data[n][:T]
#     println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
# end


# Robust Optimization

In [14]:
Γ=1.4;

In [15]:
function init_ro_variables_sub(model::Model)
    vars= Dict(
        :lambda => @variable(model, lambda[n in keys(data), t in data[n][:T]]),
        :lambda_ru => @variable(model, lambda_ru[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0),
        :lambda_rd => @variable(model, lambda_rd[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0),
        :lambda_l => @variable(model, lambda_l[n in keys(data), j in data[n][:J], t in data[n][:T]] >=0),
        :lambda_spill => @variable(model, lambda_spill[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0),
        :lambda_lc => @variable(model, lambda_lc[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]] >=0),
        :w_delta => @variable(model, w_delta[n in keys(data), q in data[n][:Q], t in data[n][:T]]),
        :w_delta_up => @variable(model, w_delta_up[n in keys(data), q in data[n][:Q], t in data[n][:T]]),
        :w_delta_down => @variable(model, w_delta_down[n in keys(data), q in data[n][:Q], t in data[n][:T]])
        )
    return vars
end;

In [16]:
function init_ro_constraints_sub(model::Model, data::Dict, vars::Dict)
    cons=Dict(
        :c_ru => @constraint(model, c_ru[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:lambda][n,t] - vars[:lambda_ru][n, i, t] <= data[n][:C_U][i]),
        :c_rd => @constraint(model, c_rd[n in keys(data), i in data[n][:I], t in data[n][:T]], -vars[:lambda][n,t] - vars[:lambda_rd][n, i, t] <= -data[n][:C_D][i]),
        :c_wspill => @constraint(model, c_wspill[n in keys(data), q in data[n][:Q], t in data[n][:T]], -vars[:lambda][n,t] -vars[:lambda_spill][n, q, t] <= data[n][:C_cur][q]),
        :c_lshed => @constraint(model, c_lshed[n in keys(data), j in data[n][:J], t in data[n][:T]], vars[:lambda][n,t] -vars[:lambda_l][n, j, t] <= data[n][:V_LOL][j]),
        #:c_lc => @constraint(model, c_lc[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], -1/data[n][:b][ℓ]*vars[:lambda][n,t] + 1/data[n][:b][ℓ]*vars[:lambda][ℓ,t] -1/data[n][:b][ℓ]*vars[:lambda_lc][n,ℓ,t] + 1/data[n][:b][ℓ]*vars[:lambda_lc][ℓ,n,t]<=0), 
        :cw1 => @constraint(model, cw1[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:w_delta][n, q, t]<= data[n][:W_d_max][q]),
        :cw2 => @constraint(model, cw2[n in keys(data), q in data[n][:Q], t in data[n][:T]], -vars[:w_delta][n, q, t]<= data[n][:W_d_max][q]),
        :cw3 => @constraint(model, cw3[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:w_delta][n, q, t] == vars[:w_delta_up][n, q, t]- vars[:w_delta_down][n, q, t]),
        :cw4 => @constraint(model, cw4, sum((vars[:w_delta_up][n, q, t] + vars[:w_delta_down][n, q, t])/ data[n][:W_d_max][q] for n in keys(data), q in data[n][:Q], t in data[n][:T]) <= Γ)
    )
    for n in keys(data)
        if n != 1
            for ℓ in data[n][:Λ]
                for t in data[n][:T]
                    #@constraint(model, -1/data[n][:b][ℓ]*vars[:lambda][n,t] + 1/data[n][:b][ℓ]*vars[:lambda][ℓ,t] -1/data[n][:b][ℓ]*vars[:lambda_lc][n,ℓ,t] + 1/data[n][:b][ℓ]*vars[:lambda_lc][ℓ,n,t] <= 0)
                    @constraint(model, 1/data[n][:b][ℓ]*vars[:lambda][n,t] - 1/data[n][:b][ℓ]*vars[:lambda][ℓ,t] <= 0)
                end
            end
        end
    end
    
    return cons
end;

In [17]:
function init_ro_objective_sub(model::Model, data::Dict, vars::Dict, vars_master::Dict)
    print("valuevars_master[:delta][1,2,1] \n")
    print(value(vars_master[:delta][1,2,1]))
    print("\nvaluevars_master[:delta][1,2,1] \n")
    print(value(vars_master[:delta][2,1,1]))
    @objective(model, Max,
    +sum(vars[:w_delta][n,q,t]*vars[:lambda][n,t] for n in keys(data), q in data[n][:Q], t in data[n][:T])
    +sum((value(vars_master[:delta][n,ℓ,t])* 1/data[n][:b][ℓ] - value(vars_master[:delta][ℓ,n,t])* 1/data[n][:b][ℓ]) * vars[:lambda][n,t] for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])
    #vars[:w_delta][n,q,t])    
    -sum(value(vars_master[:R_U][n,i,t])* vars[:lambda_ru][n,i,t] for n in keys(data), i in data[n][:I], t in data[n][:T])
    -sum(value(vars_master[:R_D][n,i,t])* vars[:lambda_rd][n,i,t] for n in keys(data), i in data[n][:I], t in data[n][:T])
    -sum(data[n][:L][j][:L][t]* vars[:lambda_l][n,j,t] for n in keys(data), j in data[n][:J], t in data[n][:T])
    -sum((data[n][:W_s][q] + vars[:w_delta][n,q,t]) * vars[:lambda_spill][n,q,t] for n in keys(data), q in data[n][:Q], t in data[n][:T])
    -sum(data[n][:LC_Max][ℓ]* vars[:lambda_lc][n,ℓ,t] for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])
    )
end;

In [18]:
function init_ro_variables_master(model::Model)

    vars= Dict(
        :alpha=> @variable(model, alpha >= 0),
        :P=> @variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #power generated from generator i
        :R_U=> @variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i
        :R_D=> @variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i
        :delta => @variable(model, delta[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]) #voltage angle
    )
    return vars
end;

In [19]:
function init_ro_constraints_master(model::Model, data::Dict, vars::Dict)
    @constraints(model, begin
        Power_balance_day_ahead[n in keys(data), t in data[n][:T]] , 
        sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(data[n][:W_s][q] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum((vars[:delta][n, ℓ, t] - vars[:delta][ℓ, n, t])* 1 / data[n][:b][ℓ]  for ℓ in data[n][:Λ])== 0
        c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
        c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
        c3[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:delta][n,ℓ, t] - vars[:delta][ℓ,n, t]) * 1 / data[n][:b][ℓ] <= data[n][:LC_Max][ℓ]
        
        Reference_Node[t in data[1][:T]], vars[:delta][1, :, t] .== 0
    end)
    
end;

In [20]:
function init_ro_objective_master(model::Model, data::Dict, vars::Dict)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T]) +
    + vars[:alpha]
    )
end;

In [21]:
#initialize master model variables
model_ro_master=Model(HiGHS.Optimizer)
vars_ro_master=init_ro_variables_master(model_ro_master)
init_ro_constraints_master(model_ro_master, data, vars_ro_master)
init_ro_objective_master(model_ro_master, data, vars_ro_master)

model_ro_sub=Model(HiGHS.Optimizer)
vars_ro_sub=init_ro_variables_sub(model_ro_sub)
cons=init_ro_constraints_sub(model_ro_sub, data, vars_ro_sub)


optimize!(model_ro_master)
#     println("\ndelta:")
#     for n in keys(data),ℓ in data[n][:Λ], t in data[n][:T]
#         println("delta[$n,$ℓ $t]: ", value(vars_ro_master[:delta][n,ℓ, t]))
#     end
    
    #initalize variables
    UB_master=0
    for n in keys(data)
    for i in data[n][:I]
        for t in data[n][:T]
            # Update UP_master using the defined n, i, and t
            UB_master += value(vars_ro_master[:P][n, i, t]) * data[n][:C][i] +
                         value(vars_ro_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                         value(vars_ro_master[:R_D][n, i, t]) * data[n][:C_RD][i]
            end
        end
    end
    print(UB_master)
    UB=0 #intial value for the cancel condition
    LB=-1
    i=1; #number of iteration (just for printing purposes)
    
    #solving the problem with Benders decomposition using dual dynamic programming – DDP

    while UB!=LB && i<15
        print("\n iteration no $i \n")

        # optimize dual subproblem with the fixed values of the masterproblem
        init_ro_objective_sub(model_ro_sub,data, vars_ro_sub, vars_ro_master)
        print("\noptimize model_ro_sub \n")
        #print(model_ro_sub)
        optimize!(model_ro_sub)
        print("\noptimize model_ro_sub \n")
        UB_master=0
        for n in keys(data)
            for i in data[n][:I]
                    for t in data[n][:T]
                        # Update UP_master using the defined n, i, and t
                        UB_master += value(vars_ro_master[:P][n, i, t]) * data[n][:C][i] +
                                     value(vars_ro_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                                     value(vars_ro_master[:R_D][n, i, t]) * data[n][:C_RD][i]
                    end
            end
        end
        UB =UB_master + objective_value(model_ro_sub)
        # add benders cut (as constraint with dual variable from the subproblem)
        objective_value(model_ro_sub)
        @constraint(model_ro_master, 
                +sum(sum(-value(vars_ro_sub[:w_delta][n,q,t]))*value(vars_ro_sub[:lambda][n,t]) for n in keys(data), q in data[n][:Q], t in data[n][:T]) 
                +sum((vars_ro_master[:delta][n,ℓ,t]* 1/data[n][:b][ℓ] - vars_ro_master[:delta][ℓ,n,t]* 1/data[n][:b][ℓ]) * value(vars_ro_sub[:lambda][n,t]) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])                
                -sum(vars_ro_master[:R_U][n,i,t]* value(vars_ro_sub[:lambda_ru][n,i,t]) for n in keys(data), i in data[n][:I], t in data[n][:T])
                -sum(vars_ro_master[:R_D][n,i,t]* value(vars_ro_sub[:lambda_rd][n,i,t]) for n in keys(data), i in data[n][:I], t in data[n][:T])
                -sum(data[n][:L][j][:L][t]* value(vars_ro_sub[:lambda_l][n,j,t]) for n in keys(data), j in data[n][:J], t in data[n][:T])
                -sum((data[n][:W_s][q] + value(vars_ro_sub[:w_delta][n,q,t])) * value(vars_ro_sub[:lambda_spill][n,q,t]) for n in keys(data), q in data[n][:Q], t in data[n][:T])
                -sum(data[n][:LC_Max][ℓ]* value(vars_ro_sub[:lambda_lc][n,ℓ,t]) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]) <= vars_ro_master[:alpha])

        # solve masterproblem with new constraint
        #print(model_ro_master)
        optimize!(model_ro_master)
        println("\nP:")
#         for n in keys(data), i in data[n][:I], t in data[n][:T]
#             println("P[$n, $i, $t]: ", JuMP.value.(vars_ro_master[:P][n, i, t]))
#         end

#         println("\nR_U:")
#         for n in keys(data), i in data[n][:I], t in data[n][:T]
#             println("R_U[$n, $i, $t]: ", value(vars_ro_master[:R_U][n,i, t]))
#         end

#         println("\nR_D:")
#         for n in keys(data), i in data[n][:I], t in data[n][:T]
#             println("R_D[$n, $i, $t]: ", value(vars_ro_master[:R_D][n,i, t]))end
#         println("\ndelta:")
#         for n in keys(data),ℓ in data[n][:Λ], t in data[n][:T]
#             println("delta[$n,$ℓ $t]: ", value(vars_ro_master[:delta][n,ℓ, t]))
#         end
        LB=objective_value(model_ro_master)
        print("LB:\n")
        print(LB)
        print("\nUP:\n")
        print(UB)
        i=i+1
    end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2328 rows, 1680 cols, 6408 nonzeros
360 rows, 768 cols, 1368 nonzeros
360 rows, 768 cols, 1368 nonzeros
Presolve : Reductions: rows 360(-2256); columns 768(-1369); elements 1368(-6072)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.7714657400e-01 Pr: 312(234107) 0s
        557     9.9578300495e+04 Pr: 0(0); Du: 0(5.32907e-15) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 557
Objective value     :  9.9578300495e+04
HiGHS run time      :          0.01
99578.30049475426
 iteration no 1 
valuevars_master[:delta][1,2,1] 
-0.0
valuevars_master[:delta][1,2,1] 
-2.555
optimize model_ro_sub 
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Iteration, Runtime, ObjVal, NullspaceDim
0, 0.008976, 0.000000, 263
652, 0.422384, 0.000000, 

In [22]:
#print(model_ro_sub)

In [23]:
#print(model_ro_master)

In [24]:
# Print the values of the variables
# println("Variable values:")
# print("alpha:")
# print(value(vars_ro_master[:alpha]))
# println("\nP:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("P[$n, $i, $t]: ", JuMP.value.(vars_ro_master[:P][n, i, t]))
# end

# println("\nR_U:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_U[$n, $i, $t]: ", value(vars_ro_master[:R_U][n,i, t]))
# end

# println("\nR_D:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_D[$n, $i, $t]: ", value(vars_ro_master[:R_D][n,i, t]))end

# println("\ndelta:")
# for n in keys(data),ℓ in data[n][:Λ], t in data[n][:T]
#     println("delta[$n,$ℓ $t]: ", value(vars_ro_master[:delta][n,ℓ, t]))
# end

In [25]:
# Print the values of the variables
println("Variable values:")

# println("\n:lambda:")
# for n in keys(data), t in data[n][:T]
#     println(":lambda[$n, $t]: ", JuMP.value.(vars_ro_sub[:lambda][n, t]))
# end

Variable values:


# Benders Decomposition with Stochastic programming

In [26]:
function init_vars_master(model::Model)
    vars= Dict(
        :P => @variable(model, P[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #energy generated,
        :R_U => @variable(model, R_U[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i,
        :R_D => @variable(model, R_D[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i,
        :W_s => @variable(model, W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0), #scheduled wind power generation at turbine q,
        :delta => @variable(model, delta[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #voltage angle,
        :PF => @variable(model, PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #power flow,
        :alpha => @variable(model, alpha >=0)
    )
    return vars
end;

In [27]:
function init_vars_sub(model::Model)
    vars= Dict(
        :r_U => @variable(model, r_U[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0), #up regulation of generator i in case 𝜔
        :r_D => @variable(model, r_D[n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]] >=0), #down regulation of generator i in case 𝜔
        :L_Shed => @variable(model, L_Shed[n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]] >=0), #loss of load at load demand j in case 𝜔
        :W_spill => @variable(model, W_spill[n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]] >=0), #curtailment of turbine q in case 𝜔
        :delta_scenario => @variable(model, delta_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios),t in data[n][:T]]), #voltage angle in case 𝜔
        :PF_scenario => @variable(model, PF_scenario[n in keys(data), ℓ in data[n][:Λ], ω in keys(scenarios), t in data[n][:T]]), #power flow in case 𝜔
        
        :W_s_sub => @variable(model, W_s_sub[n in keys(data), q in data[n][:Q], t in data[n][:T]] >=0), #scheduled wind power generation at turbine q
        :PF_sub => @variable(model, PF_sub[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]]), #power flow
        :R_U_sub => @variable(model, R_U_sub[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed upward reserve capacity of generator i,
        :R_D_sub => @variable(model, R_D_sub[n in keys(data), i in data[n][:I], t in data[n][:T]] >=0), #committed downward reserve capacity of generator i,
    )
    return vars
end;

In [28]:
# Initializes constraints
function init_cons_master(model::Model, data::Dict, vars::Dict)
    @constraints(model, begin 
    c1[n in keys(data),i in data[n][:I], t in data[n][:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[n][:P_max][i]
    
    c2[n in keys(data),i in data[n][:I] ,t in data[n][:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
    c5[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF][n, ℓ, t] <= data[n][:LC_Max][ℓ]
    
    c6[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:delta][n,ℓ, t] - vars[:delta][ℓ,n, t]) * 1 / data[n][:b][ℓ]  == vars[:PF][n, ℓ, t]
       
    Reference_Node[t in data[1][:T]], vars[:delta][1, :, t] .== 0
            
    Power_balance_day_ahead[n in keys(data), t in data[n][:T]], 
    sum(vars[:P][n,i,t] for i in data[n][:I]) + sum(vars[:W_s][n,q,t] for q in data[n][:Q]) - sum(data[n][:L][j][:L][t] for j in data[n][:J]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[n][:Λ]) == 0

    end)
end;

In [29]:
# Initializes constraints
function init_cons_sub(model::Model, data::Dict, vars::Dict, vars_master::Dict)
    cons= Dict(

    :c3 => @constraint(model,c3[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_U][n, i, ω, t] <= vars[:R_U_sub][n,i,t]),
    :c4 => @constraint(model,c4[n in keys(data),i in data[n][:I], t in data[n][:T], ω in keys(scenarios)], vars[:r_D][n, i, ω, t] <= vars[:R_D_sub][n,i,t]),
    :c5_scenario => @constraint(model,c5_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_scenario][n, ℓ, ω, t] <= data[n][:LC_Max][ℓ]),
    
    :c6_scednario => @constraint(model,c6_scenario[ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], (vars[:delta_scenario][n, ℓ, ω, t] - vars[:delta_scenario][ℓ,n, ω, t]) * 1 / data[n][:b][ℓ] == vars[:PF_scenario][n, ℓ, ω, t]),
       
    
    :c9 => @constraint(model,c9[ω in keys(scenarios), n in keys(data),t in data[n][:T], q in data[n][:Q]], vars[:W_spill][n,q,ω,t] <= data[n][:wind_energy][q][ω][:W_realized][t]),
    :c10 => @constraint(model,c10[ω in keys(scenarios), n in keys(data),t in data[n][:T], j in data[n][:J]], vars[:L_Shed][n,j, ω, t] <= data[n][:L][j][:L][t]),
    
    :c11 => @constraint(model, Reference_Node_Scenario[ω in keys(scenarios), t in data[1][:T]], vars[:delta_scenario][1, :, ω, t] .== 0),

    :Power_balance_at_stage => @constraint(model,Power_balance_at_stage[ω in keys(scenarios), n in keys(data), t in data[n][:T]], 
    sum(vars[:r_U][n,i, ω, t] for i in data[n][:I]) -
    sum(vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
    sum(vars[:L_Shed][n,j, ω, t] for j in data[n][:J]) +
    sum(data[n][:wind_energy][q][ω][:W_realized][t] -vars[:W_s_sub][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[n][:Q]) -
    sum(vars[:PF_scenario][n, ℓ, ω, t] for ℓ in data[n][:Λ]) +
    sum(vars[:PF_sub][n,ℓ,t] for ℓ in data[n][:Λ]) == 0),
    
    :η_RU => @constraint(model,η_RU[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:R_U_sub][n,i,t] == value(vars_master[:R_U][n,i,t])),
    :η_RD => @constraint(model,η_RD[n in keys(data), i in data[n][:I], t in data[n][:T]], vars[:R_D_sub][n,i,t] == value(vars_master[:R_D][n,i,t])),
    :η_W_s => @constraint(model,η_W_s[n in keys(data), q in data[n][:Q], t in data[n][:T]], vars[:W_s_sub][n,q,t] == value(vars_master[:W_s][n,q,t])),
    :η_PF => @constraint(model,η_PF[n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]], vars[:PF_sub][n,ℓ,t] == value(vars_master[:PF][n, ℓ, t]))
    )
end;

In [30]:
# Initializes objective function master
function init_obj_master(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[n][:C][i] + vars[:R_U][n,i,t] * data[n][:C_RU][i] + vars[:R_D][n,i,t] * data[n][:C_RD][i] for n in keys(data), i in data[n][:I], t in data[n][:T])
    + vars[:alpha]
    )
end;

In [31]:
# Initializes objective function sub
function init_obj_sub(model, data, vars)
    @objective(model, Min, sum(scenarios[ω][:π] * (sum(data[n][:C_U][i] * vars[:r_U][n,i, ω ,t] - data[n][:C_D][i] * vars[:r_D][n,i, ω, t] for i in data[n][:I]) +
                sum(data[n][:C_cur][q] * vars[:W_spill][n,q, ω, t] for q in data[n][:Q]) +
                sum(data[n][:V_LOL][j] * vars[:L_Shed][n, j, ω, t] for j in data[n][:J])) for n in keys(data), ω in keys(scenarios), t in data[n][:T])
    )
end;

bd_master=Model(HiGHS.Optimizer)
vars_master=init_vars_master(bd_master)
init_cons_master(bd_master, data, vars_master)
init_obj_master(bd_master, data, vars_master)
print(bd_master)
#optimize!(bd_master)

#print(value(vars_master[:W_s][1,1.0,1]))
bd_sub=Model(GLPK.Optimizer)
#bd_sub=Model(HiGHS.Optimizer)
vars_sub=init_vars_sub(bd_sub)
#cons_sub=init_cons_sub(bd_sub, data, vars_sub, vars_master)
#init_obj_sub(bd_sub, data, vars_sub)
#optimize!(bd_sub)
#print(objective_value(bd_sub))
print(bd_sub)

In [32]:
#optimize!(bd_sub)
#obj_value = objective_value(bd_sub)
#print(obj_value)
#for n in keys(data)
#    for q in data[n][:Q]
#        for t in data[n][:T]
#            # Update UP_master using the defined n, i, and t
#            print(dual(cons_sub[:η_W_s][n,q,t]))
#        end
#    end
#end

In [33]:
function benders_decomp_st(master::Model, data::Dict, vars_master::Dict)
    #initialize master model variables
    optimize!(master)
    
    #init sub cons and objective
    
    #initalize variables
    UB_master=0
    for n in keys(data)
    for i in data[n][:I]
        for t in data[n][:T]
            # Update UP_master using the defined n, i, and t
            UB_master += value(vars_master[:P][n, i, t]) * data[n][:C][i] +
                         value(vars_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                         value(vars_master[:R_D][n, i, t]) * data[n][:C_RD][i]
            end
        end
    end
    print(UB_master)
    UB=0 #intial value for the cancel condition
    LB=-1
    i=1; #number of iteration (just for printing purposes)
    
    #solving the problem with Benders decomposition using dual dynamic programming – DDP

    while UB!=LB && i<2
        print("\n iteration no $i \n")
        
        sub=Model(GLPK.Optimizer)
        vars_sub=init_vars_sub(sub)
        cons_sub=init_cons_sub(sub, data, vars_sub, vars_master)
        init_obj_sub(sub, data, vars_sub)
        #print(sub)

        # optimize dual subproblem with the fixed values of the masterproblem
        optimize!(sub)
        obj_sub=objective_value(sub)
        print("\n obj_sub \n")
        #print(sub)
        #print(obj_sub)
        # Print the values of the variables
        


#         println("\nL_Shed:")
#         for n in keys(data), j in data[n][:J], ω in keys(scenarios), t in data[n][:T]
#             println("L_Shed[$n, $j, $ω, $t]: ", value(vars_sub[:L_Shed][n,j, ω, t]))
#         end

#         println("\nW_spill:")
#         for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
#             println("W_spill[$n, $q, $ω, $t]: ", value(vars_sub[:W_spill][n,q, ω, t]))
#         end

#         println("\nPF_scenario[n, ℓ, ω, t]:")
#         for ω in keys(scenarios), n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
#             println("PF_scenario[$n, $ℓ, $ω, $t]: ", value(vars_sub[:PF_scenario][n, ℓ, ω, t]))
#         end

#         println("\nr_U[n,i, ω, t]:")
#         for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#             println("r_U[$n, $i, $ω, $t]: ", value(vars_sub[:r_U][n,i, ω, t]))
#         end

#         println("\nr_D[n,i, ω, t]:")
#         for n in keys(data), i in data[n][:I], ω in keys(scenarios), t in data[n][:T]
#             println("r_D[$n, $i, $ω, $t]: ", value(vars_sub[:r_D][n,i, ω, t]))
#         end

        
        # add benders cut (as constraint with dual variable from the subproblem)
        @constraint(master, obj_sub
            +sum(dual(cons_sub[:η_W_s][n,q,t])*(vars_master[:W_s][n, q, t]- value(vars_master[:W_s][n, q, t])) for n in keys(data), q in data[n][:Q], t in data[n][:T]) 
            + sum(dual(cons_sub[:η_RD][n,i,t])* (vars_master[:R_D][n, i, t]- value(vars_master[:R_D][n, i, t])) for n in keys(data), i in data[n][:I], t in data[n][:T]) 
            + sum(dual(cons_sub[:η_RU][n,i,t])* (vars_master[:R_U][n, i, t]- value(vars_master[:R_U][n, i, t])) for n in keys(data), i in data[n][:I], t in data[n][:T])
            + sum(dual(cons_sub[:η_PF][n,ℓ,t])* (vars_master[:PF][n, ℓ, t]- value(vars_master[:PF][n, ℓ, t])) for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T])<= vars_master[:alpha]
        )
        # solve masterproblem with new constraint
        optimize!(master)
        LB=objective_value(master)
        UB=0
        for n in keys(data)
            for i in data[n][:I]
                    for t in data[n][:T]
                        # Update UP_master using the defined n, i, and t
                        UB += value(vars_master[:P][n, i, t]) * data[n][:C][i] +
                                     value(vars_master[:R_U][n, i, t]) * data[n][:C_RU][i] +
                                     value(vars_master[:R_D][n, i, t]) * data[n][:C_RD][i]
                    end
            end
        end
        UB =UB + objective_value(sub)
        print(UB)
        print("LB:\n")
        print(LB)
        print("\nUP:\n")
        print(UB)
        i=i+1
    end
    
end;

In [34]:
bd_master=Model(HiGHS.Optimizer)
vars_master=init_vars_master(bd_master)
init_cons_master(bd_master, data, vars_master)
init_obj_master(bd_master, data, vars_master)

bd_sub=Model(GLPK.Optimizer)
vars_sub=init_vars_sub(bd_sub)
#print(bd_sub)

benders_decomp_st(bd_master, data, vars_master)



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2424 rows, 3480 cols, 6888 nonzeros
456 rows, 1080 cols, 1776 nonzeros
432 rows, 1008 cols, 1680 nonzeros
Presolve : Reductions: rows 432(-3816); columns 1008(-3001); elements 1680(-7632)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.1567396022e-01 Pr: 408(282172) 0s
        649     0.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 649
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.02
0.0
 iteration no 1 

 obj_sub 
Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     5.6371010673e-01 Pr: 1(6346.81); Du: 0(9.64843e-08) 0s
         80     0.0000000000e+00 Pr: 0(0) 0s
Model   status      : Optimal

In [35]:
cons_sub=init_cons_sub(bd_sub, data, vars_sub, vars_master)
init_obj_sub(bd_sub, data, vars_sub)
#print(bd_sub)

W_spill[5,2.0,5,1] + 20 L_Shed[5,5.0,5,1] + W_spill[5,2.0,5,2] + 20 L_Shed[5,5.0,5,2] + W_spill[5,2.0,5,3] + 20 L_Shed[5,5.0,5,3] + W_spill[5,2.0,5,4] + 20 L_Shed[5,5.0,5,4] + W_spill[5,2.0,5,5] + 20 L_Shed[5,5.0,5,5] + W_spill[5,2.0,5,6] + 20 L_Shed[5,5.0,5,6] + W_spill[5,2.0,5,7] + 20 L_Shed[5,5.0,5,7] + W_spill[5,2.0,5,8] + 20 L_Shed[5,5.0,5,8] + W_spill[5,2.0,5,9] + 20 L_Shed[5,5.0,5,9] + W_spill[5,2.0,5,10] + 20 L_Shed[5,5.0,5,10] + W_spill[5,2.0,5,11] + 20 L_Shed[5,5.0,5,11] + W_spill[5,2.0,5,12] + 20 L_Shed[5,5.0,5,12] + W_spill[5,2.0,5,13] + 20 L_Shed[5,5.0,5,13] + W_spill[5,2.0,5,14] + 20 L_Shed[5,5.0,5,14] + W_spill[5,2.0,5,15] + 20 L_Shed[5,5.0,5,15] + [[...8820 terms omitted...]] + W_spill[21,9.0,3,19] + W_spill[21,9.0,3,20] + W_spill[21,9.0,3,21] + W_spill[21,9.0,3,22] + W_spill[21,9.0,3,23] + W_spill[21,9.0,3,24] + W_spill[21,9.0,1,1] + W_spill[21,9.0,1,2] + W_spill[21,9.0,1,3] + W_spill[21,9.0,1,4] + W_spill[21,9.0,1,5] + W_spill[21,9.0,1,6] + W_spill[21,9.0,1,7] + W_spi

In [36]:
# print(bd_master)

# # Print the values of the variables
# println("Variable values:")
# println("P:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("P[$n, $i, $t]: ", JuMP.value.(vars_master[:P][n, i, t]))
# end

# println("\nR_U:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_U[$n, $i, $t]: ", value(vars_master[:R_U][n,i, t]))
# end

# println("\nR_D:")
# for n in keys(data), i in data[n][:I], t in data[n][:T]
#     println("R_D[$n, $i, $t]: ", value(vars_master[:R_D][n,i, t]))end


# println("\nW_s:")
# for n in keys(data), q in data[n][:Q], t in data[n][:T]
#     println("W_s[$n, $q, $t]: ", value(vars_master[:W_s][n,q, t]))
# end

# println("\nPF[n, ℓ, t]:")
# for n in keys(data), ℓ in data[n][:Λ], t in data[n][:T]
#     println("PF[$n, $ℓ, $t]: ", value(vars_master[:PF][n, ℓ, t]))
# end

# println("\nData values:")

# println("\nW_realized:")
# for n in keys(data), q in data[n][:Q], ω in keys(scenarios), t in data[n][:T]
#     println("W_realized[$n, $q, $ω, $t]: ", value(data[n][:Windenergy][q][ω][:W_realized][t]))
# end

# println("\nLoad:")
# for n in keys(data), j in data[n][:J], t in data[n][:T]
#     println("L[$n, $j, $t]:", value(data[n][:L][j][:L][t]))
# end